In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

import time

In [ ]:
def get_episode_information(stage, act):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    url = "https://initiald.fandom.com/wiki/"

    act_url = f"{url}{stage}_-_Act_{act}"
    req = requests.get(act_url, headers=headers).text

    info = {}
    info["stage"] = stage
    info["act"] = act

    synopsis = req[req.find('<span class="mw-headline" id="Synopsis">'): req.find('<span class="mw-headline" id="Plot">')]
    synopsis = BeautifulSoup(synopsis, "html.parser").find("p").text

    info["synopsis"] = synopsis

    plot = req[req.find('<span class="mw-headline" id="Plot">'): req.find('<span class="mw-headline" id="Stats">')]
    plot = BeautifulSoup(plot, "html.parser").find_all("p")
    plot = "".join([i.text for i in plot])

    info["plot"] = plot

    airdate = req[req.find('<span class="mw-headline" id="Airdates">'): req.find('<span id="Characters_(in_order_of_appearance)">')]
    airdate = BeautifulSoup(airdate, "html.parser").find("li").text
    
    info["airdate"] = airdate

    characters = req[req.find('<span class="mw-headline" id="Char'): req.find('<span class="mw-headline" id="Cars">'): ]
    characters = BeautifulSoup(characters, "html.parser").find_all("li")
    characters = [i.text for i in characters]

    info["characters"] = characters

    cars = req[req.find('<span class="mw-headline" id="Cars">'): req.find('<span class="mw-headline" id="Music">'): ]
    cars = BeautifulSoup(cars, "html.parser").find_all("a")
    cars = [i.text for i in cars]

    info["cars"] = cars

    music = req[req.find('<span class="mw-headline" id="Music">'): req.find('<span class="mw-headline" id="Quotes">'): ]
    music = BeautifulSoup(music, "html.parser").find_all("a")
    music = [i.text for i in music]

    info["music"] = music

    # quotes = req[req.find('<span class="mw-headline" id="Quotes">'): req.find('<span class="mw-headline" id="Notes_.26_Trivia">')]
    # quotes = req[req.find('<span class="mw-headline" id="Quot'): req.find('<span class="mw-headline" id="N'): ]
    # quotes = [i.text for i in quotes]

    # info["quotes"] = quotes

    return info

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
url = "https://initiald.fandom.com/wiki/"
stages = {
    "First_Stage": 26,
    "Second_Stage": 13,
    "Fourth_Stage": 24,
    "Fifth_Stage": 14,
    "Final_Stage": 4,
    }

rows = []
for part in stages:
    act = 1
    while act <= stages[part]:
        rows.append(get_episode_information(stage=part, act=act))
        time.sleep(1.5)
        print(f"Done: {part} Act: {act}")
        act += 1

In [ ]:
import pandas as pd

initial_d = pd.DataFrame(rows)
initial_d.head()
initial_d.to_csv("fandom.csv", index=False)

In [ ]:
# Car Counts
initial_d[["stage", "act", "cars"]].explode("cars")["cars"].value_counts()

In [ ]:
# Create adjacency matrix for cars and stages

import networkx as nx
import numpy as np

G = nx.MultiDiGraph()

for i in initial_d["stage"].unique():
    G.add_node(i)
for i in initial_d["cars"].unique():
    G.add_node(i)

for row in initial_d.itertuples():
    G.add_edge(row.stage, row.cars)
    G.add_edge(row.cars, row.stage)

zeros = np.zeros((10, 10))
top = np.concatenate([zeros, a], axis=1)
bottom = np.concatenate([a, zeros], axis=1)
np.concatenate([top, bottom], axis=0)